In [4]:
import os
from dotenv import load_dotenv
import pandas as pd

pd.set_option('display.max_columns', None)
load_dotenv()

endpoint   = os.getenv("MINIO_ENDPOINT")
bucket     = os.getenv("MINIO_BUCKET")
parquet    = os.getenv("MINIO_PARQUET_PATH")
access_key = os.getenv("MINIO_ACCESS_KEY")
secret_key = os.getenv("MINIO_SECRET_KEY")

storage_opts = {
    "key": access_key,
    "secret": secret_key,
    "client_kwargs": {"endpoint_url": endpoint},
}

# Exemplo: abrir o parquet de 2025
path = f"s3://{bucket}/{parquet}/add_discentes.parquet"
df = pd.read_parquet(path, storage_options=storage_opts)

In [5]:
df.columns

Index(['AN_BASE', 'CD_PROGRAMA_IES', 'NM_PROGRAMA_IES', 'CD_ENTIDADE_CAPES',
       'SG_ENTIDADE_ENSINO', 'NM_ENTIDADE_ENSINO', 'CS_STATUS_JURIDICO',
       'DS_DEPENDENCIA_ADMINISTRATIVA', 'CD_GRANDE_AREA_CONHECIMENTO',
       'NM_GRANDE_AREA_CONHECIMENTO', 'CD_AREA_CONHECIMENTO',
       'NM_AREA_CONHECIMENTO', 'CD_SUBAREA_CONHECIMENTO',
       'NM_SUBAREA_CONHECIMENTO', 'CD_ESPECIALIDADE', 'NM_ESPECIALIDADE',
       'CD_AREA_AVALIACAO', 'NM_AREA_AVALIACAO', 'CD_OCDE_AREA_ESPECIFICA',
       'NM_OCDE_AREA_ESPECIFICA', 'CD_OCDE_AREA_GERAL', 'NM_OCDE_AREA_GERAL',
       'CD_GRAU_PROGRAMA', 'NM_GRAU_PROGRAMA', 'SG_UF_PROGRAMA',
       'CD_IBGE_PROGRAMA_IES', 'NM_MUNICIPIO_PROGRAMA_IES', 'ID_DISCENTE',
       'TP_DOCUMENTO_DISCENTE', 'NR_DOCUMENTO_DISCENTE', 'ID_PESSOA',
       'NM_DISCENTE', 'ST_INGRESSANTE', 'NM_PAIS_NACIONALIDADE_DISCENTE',
       'DS_TIPO_NACIONALIDADE_DISCENTE', 'TP_SEXO_DISCENTE',
       'DT_NASCIMENTO_DISCENTE', 'DS_IDADE_ANOBASE',
       'DS_GRAU_ACADEMICO_DISCENT

In [6]:
df

,AN_BASE,CD_PROGRAMA_IES,NM_PROGRAMA_IES,CD_ENTIDADE_CAPES,SG_ENTIDADE_ENSINO,NM_ENTIDADE_ENSINO,CS_STATUS_JURIDICO,DS_DEPENDENCIA_ADMINISTRATIVA,CD_GRANDE_AREA_CONHECIMENTO,NM_GRANDE_AREA_CONHECIMENTO,CD_AREA_CONHECIMENTO,NM_AREA_CONHECIMENTO,CD_SUBAREA_CONHECIMENTO,NM_SUBAREA_CONHECIMENTO,CD_ESPECIALIDADE,NM_ESPECIALIDADE,CD_AREA_AVALIACAO,NM_AREA_AVALIACAO,CD_OCDE_AREA_ESPECIFICA,NM_OCDE_AREA_ESPECIFICA,CD_OCDE_AREA_GERAL,NM_OCDE_AREA_GERAL,CD_GRAU_PROGRAMA,NM_GRAU_PROGRAMA,SG_UF_PROGRAMA,CD_IBGE_PROGRAMA_IES,NM_MUNICIPIO_PROGRAMA_IES,ID_DISCENTE,TP_DOCUMENTO_DISCENTE,NR_DOCUMENTO_DISCENTE,ID_PESSOA,NM_DISCENTE,ST_INGRESSANTE,NM_PAIS_NACIONALIDADE_DISCENTE,DS_TIPO_NACIONALIDADE_DISCENTE,TP_SEXO_DISCENTE,DT_NASCIMENTO_DISCENTE,DS_IDADE_ANOBASE,DS_GRAU_ACADEMICO_DISCENTE,DT_MATRICULA_DISCENTE,DT_SITUACAO_DISCENTE,NM_SITUACAO_DISCENTE,DI_MATRICULA_DISCENTE,ME_MATRICULA_DISCENTE,AN_MATRICULA_DISCENTE,DI_SITUACAO_DISCENTE,ME_SITUACAO_DISCENTE,AN_SITUACAO_DISCENTE,DS_FAIXA_ETARIA,CD_CONCEITO_CURSO,CD_CONCEITO_PROGRAMA,NM_MODALIDADE_PROGRAMA,NM_ORIENTADOR_PRINCIPAL,NM_TESE_DISSERTACAO,NR_CNPJ_IES,NR_DOC_ORIENTADOR_PRINCIPAL,QT_MES_TITULACAO,NM_REGIAO,ID_ADD_DISCENTE,ID_ADD_FOTO_PROGRAMA_IES,ID_LATTES,ID_ADD_FOTO_PROGRAMA,NR_CNPJ_MANTENEDORA,ID_PESSOA_ORIGINAL,DT_CARGA,NM_RACA_COR,IN_NECESSIDADE_PESSOAL,CD_ENTIDADE_EMEC
0,2013,40020010005P2,CULTURA E SOCIEDADE: DIÁLOGOS INTERDISCIPLINARES,40020010,UTP,UNIVERSIDADE TUIUTI DO PARANÁ,PARTICULAR,PRIVADA,90000005,MULTIDISCIPLINAR,90100000,INTERDISCIPLINAR,90192000,SOCIAIS E HUMANIDADES,NA,NÃO SE APLICA,45,INTERDISCIPLINAR,86.0,SERVIÇOS DE SEGURANÇA,8.0,SERVIÇOS,1,MESTRADO,PR,4106902,CURITIBA,441309,CPF,00554681951,731664,ALEXANDRE ALVES ARANHA,NÃO,BRASIL,BRASILEIRO,MASCULINO,1978-10-23,35,MESTRADO,2011-08-01,2013-11-21,TITULADO,1,8,2011,21,11,2013,35 A 39 ANOS,3,3,ACADÊMICO,RAFAEL TASSI TEIXEIRA,CULTURAS URBANAS: SIMBOLOGIAS E SIGNIFICADOS,NI,03119677957,27.0,SUL,85125.0,20932,6058282328829849,2533.0,76590249000166,731664,2018-11-09 16:33:43,None,None,None
1,2013,40020010005P2,CULTURA E SOCIEDADE: DIÁLOGOS INTERDISCIPLINARES,40020010,UTP,UNIVERSIDADE TUIUTI DO PARANÁ,PARTICULAR,PRIVADA,90000005,MULTIDISCIPLINAR,90100000,INTERDISCIPLINAR,90192000,SOCIAIS E HUMANIDADES,NA,NÃO SE APLICA,45,INTERDISCIPLINAR,86.0,SERVIÇOS DE SEGURANÇA,8.0,SERVIÇOS,1,MESTRADO,PR,4106902,CURITIBA,441302,CPF,59970073915,730249,ANA CRISTINA FABIANOVICZ,NÃO,BRASIL,BRASILEIRO,FEMININO,1966-08-24,47,MESTRADO,2011-08-01,2013-05-16,TITULADO,1,8,2011,16,5,2013,45 A 49 ANOS,3,3,ACADÊMICO,GERALDO MAGELA PIERONE,A CULTURA INSTITUCIONAL NAS UNIDADES SOCIOEDUC...,NI,18697615615,21.0,SUL,85129.0,20932,7238496939620783,2533.0,76590249000166,730249,2018-11-09 16:33:43,None,None,None
2,2013,40020010005P2,CULTURA E SOCIEDADE: DIÁLOGOS INTERDISCIPLINARES,40020010,UTP,UNIVERSIDADE TUIUTI DO PARANÁ,PARTICULAR,PRIVADA,90000005,MULTIDISCIPLINAR,90100000,INTERDISCIPLINAR,90192000,SOCIAIS E HUMANIDADES,NA,NÃO SE APLICA,45,INTERDISCIPLINAR,86.0,SERVIÇOS DE SEGURANÇA,8.0,SERVIÇOS,1,MESTRADO,PR,4106902,CURITIBA,441300,CPF,51450763987,731543,CARLOS ELIAS MATALON,NÃO,BRASIL,BRASILEIRO,MASCULINO,1962-02-22,51,MESTRADO,2011-08-01,2011-08-01,MATRICULADO,1,8,2011,1,8,2011,50 A 54 ANOS,3,3,ACADÊMICO,NÃO SE APLICA,NÃO SE APLICA,NI,NÃO SE APLICA,0.0,SUL,85132.0,20932,8458183079173765,2533.0,76590249000166,731543,2018-11-09 16:33:43,None,None,None
3,2013,40020010005P2,CULTURA E SOCIEDADE: DIÁLOGOS INTERDISCIPLINARES,40020010,UTP,UNIVERSIDADE TUIUTI DO PARANÁ,PARTICULAR,PRIVADA,90000005,MULTIDISCIPLINAR,90100000,INTERDISCIPLINAR,90192000,SOCIAIS E HUMANIDADES,NA,NÃO SE APLICA,45,INTERDISCIPLINAR,86.0,SERVIÇOS DE SEGURANÇA,8.0,SERVIÇOS,1,MESTRADO,PR,4106902,CURITIBA,441301,CPF,04884458923,731501,CASSIA CRISTINA MORETTO DA SILVA,NÃO,BRASIL,BRASILEIRO,FEMININO,1984-04-24,29,MESTRADO,2011-08-01,2013-07-31,TITULADO,1,8,2011,31,7,2013,25 A 29 ANOS,3,3,ACADÊMICO,PEDRO LEAO DA COSTA NETO,RELAÇÕES DE TRABALHO E PÓS-MODERNIDADE: UM EST...,NI,47935480915,23.0

In [7]:
# Verificar informações sobre o DataFrame
print(f"Número de linhas: {len(df)}")
print(f"Número de colunas: {len(df.columns)}")
print(f"Forma (shape): {df.shape}")
print(f"DataFrame vazio: {df.empty}")

Número de linhas: 4994134
Número de colunas: 68
Forma (shape): (4994134, 68)
DataFrame vazio: False
